# RAG avec le service Azure AI Agent et Semantic Kernel

Cet extrait de code montre comment créer et gérer un agent Azure AI pour la génération augmentée par récupération (RAG) en utilisant le `Azure AI Agent Service` et `Semantic Kernel`. L'agent traite les requêtes des utilisateurs en fonction du contexte récupéré et fournit des réponses précises en conséquence.


Correction de la version SQLite  
Si vous rencontrez l'erreur :  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Décommentez ce bloc de code au début de votre notebook :  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importation des packages
Le code suivant importe les packages nécessaires :


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Génération augmentée par récupération avec Semantic Kernel et Azure AI Agent Service

Cet exemple illustre comment utiliser le **Azure AI Agent Service** pour effectuer une **génération augmentée par récupération (RAG)** en combinant un modèle de langage avec un contexte spécifique issu d'un document téléchargé.

### Comment ça fonctionne

1. **Téléchargement du document** : Un fichier markdown (document.md) contenant des informations (la police d'assurance voyage de Contoso) est téléchargé dans le service d'agent.

2. **Création d'un magasin de vecteurs** : Le document est indexé dans un magasin de vecteurs pour permettre une recherche sémantique dans son contenu.

3. **Configuration de l'agent** : Un agent est configuré en utilisant le modèle `gpt-4o` avec les instructions strictes suivantes :
   - Répondre uniquement aux questions en se basant sur le contenu récupéré du document.
   - Refuser de répondre si la question est hors de portée.

4. **Intégration de l'outil de recherche de fichiers** : Le `FileSearchTool` est enregistré avec l'agent, permettant au modèle de rechercher et de récupérer des extraits pertinents du document indexé lors de l'inférence.

5. **Interaction avec l'utilisateur** : Les utilisateurs peuvent poser des questions. Si des informations pertinentes sont trouvées dans le document, l'agent génère une réponse fondée.  
   Sinon, l'agent répond explicitement que le document ne contient pas suffisamment d'informations.


### Fonction Principale

La fonction principale est le point d'entrée de votre programme. Elle est généralement utilisée pour orchestrer l'exécution des différentes parties de votre code.

#### Exemple de code

Voici un exemple de fonction principale simple :

```python
def main():
    print("Bienvenue dans le programme !")
    # Appeler d'autres fonctions ici
```

#### Pourquoi utiliser une fonction principale ?

- **Clarté** : Elle permet de structurer votre code de manière logique.
- **Réutilisabilité** : Vous pouvez facilement réutiliser ou tester des parties de votre code.
- **Gestion des erreurs** : Facilite l'ajout de gestion des exceptions au niveau global.

#### Bonnes pratiques

- Nommez toujours votre fonction principale `main` pour respecter les conventions.
- Utilisez `if __name__ == "__main__":` pour exécuter la fonction principale uniquement lorsque le fichier est exécuté directement.
- Évitez de surcharger la fonction principale avec trop de logique. Divisez votre code en fonctions plus petites et spécifiques.


Assurez-vous d'abord d'exécuter `az login` en utilisant l'Azure CLI afin que le contexte d'authentification approprié soit fourni lors de l'utilisation de `DefaultAzureCredential`. Le service Azure AI Agent ne utilise pas de clés API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

# Règles importantes

Ce fichier contient des informations essentielles pour comprendre comment utiliser cette documentation. Veuillez lire attentivement les sections suivantes.

## Introduction

Bienvenue dans notre guide utilisateur ! Ce document a été conçu pour vous aider à naviguer dans les fonctionnalités principales de notre produit. Si vous êtes nouveau, commencez par la section [Premiers pas](../../../../05-agentic-rag/code_samples).

## Premiers pas

Pour commencer, suivez ces étapes simples :

1. Installez le logiciel en suivant les instructions sur @@INLINE_CODE_0@@.
2. Configurez votre environnement en modifiant le fichier de configuration situé à @@INLINE_CODE_1@@.
3. Lancez l'application en exécutant la commande suivante :  
   ```bash
   ```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```
   ```

[!NOTE] Assurez-vous d'avoir les permissions nécessaires pour exécuter cette commande.

## Résolution des problèmes

Si vous rencontrez des problèmes, consultez la section [FAQ](../../../../05-agentic-rag/code_samples) ou essayez les solutions suivantes :

- **Problème :** L'application ne démarre pas.  
  **Solution :** Vérifiez que toutes les dépendances sont correctement installées en exécutant :  
  ```bash
  @@CODE_BLOCK_1@@
  ```

- **Problème :** Erreur de connexion à la base de données.  
  **Solution :** Assurez-vous que les paramètres de connexion dans @@INLINE_CODE_2@@ sont corrects.

[!WARNING] Ne modifiez pas les fichiers système sans sauvegarde préalable.

## FAQ

### Comment réinitialiser mon mot de passe ?

Pour réinitialiser votre mot de passe, suivez les étapes décrites dans @@INLINE_CODE_3@@.

### Où puis-je trouver les journaux d'erreurs ?

Les journaux d'erreurs sont situés dans le répertoire suivant :  
```bash
@@CODE_BLOCK_2@@
```

[!TIP] Utilisez un outil comme @@INLINE_CODE_4@@ pour analyser les journaux plus facilement.

## Support

Si vous avez besoin d'aide supplémentaire, contactez notre équipe de support via @@INLINE_CODE_5@@.

[!IMPORTANT] N'oubliez pas de fournir votre numéro de version et une description détaillée du problème lorsque vous contactez le support.

Merci d'utiliser notre produit ! Nous espérons que ce guide vous sera utile.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
